In [7]:
import pandas as pd

root = "C:\\Users\\user\\SadafPythonCode\\MLHackathon\\ML-for-Good-Hackathon\\Data\\"
df = pd.read_csv(root + 'CrisisLogger\\crisislogger.csv')
a = df.iloc[28,1]
a

"It has been very challenges for us. In the beginning of March, I have been diagnosed with breast cancer. Cancer diagnosis along is lonely and frustrating, especially if you are under 40 with kids ( I feel I am the youngest one in cancer center) and then COVID-19 happen. My treatment plan got impacted but at least I am getting some treatment and doctors are doing everything to help me. Additionally, my mom's knee replacement surgery got postponed and she is now can only jump on 1 leg... My FIL surgery to reverse colostomy bag after his cancer treatment is post postponed and we cannot see my in laws as we are afraid of COVID. My 5th grader will miss his graduation and misses his friends who will be going to different school next year. My preschooler is very active and I am thankful that we have the backyard. So it is a lot to handle. On top of it, both my hubby and I are working from home now. We do not have private office at home and since we are working for non profit international or

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

# function for rule 2
def rule2(text):    
    doc = nlp(text)
    pat = []    
    # iterate over tokens
    for token in doc:
        phrase = ''
        # if the word is a subject noun or an object noun
        if (token.pos_ == 'NOUN')\
            and (token.dep_ in ['dobj','pobj','nsubj','nsubjpass']):
            # iterate over the children nodes
            for subtoken in token.children:
                # if word is an adjective or has a compound dependency
                if (subtoken.pos_ == 'ADJ') or (subtoken.dep_ == 'compound'):
                    phrase += subtoken.text + ' '                    
            if len(phrase)!=0:
                phrase += token.text             
        if  len(phrase)!=0:
            pat.append(phrase)
    return pat


### Noun Phrases

In [9]:
rule2(a) # noun phrases

['breast cancer',
 'Cancer diagnosis',
 'cancer center',
 'treatment plan',
 'replacement surgery',
 'FIL surgery',
 'colostomy bag',
 'cancer treatment',
 '5th grader',
 'different school',
 'private office',
 'non international organization',
 'work load',
 'big family',
 'zoom meetings',
 'zoomba classes',
 '5th grader',
 'movie nights',
 'stars nights',
 'best care',
 'mommy game',
 'Saturday school',
 'extra activities',
 'hip hop classes']

### I/We phrases

In [10]:
def rule2_mod(text,index):
    
    doc = nlp(text)

    phrase = ''
    
    for token in doc:
        
        if token.i == index:
            
            for subtoken in token.children:
                if (subtoken.pos_ == 'ADJ'):
                    phrase += ' '+subtoken.text
            break
    
    return phrase

# rule 1 modified function
def rule1_mod(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:
        # root word
        if (token.pos_=='VERB'):
            
            phrase =''
            
            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    
                    # look for subject modifier
                    adj = rule2_mod(text,sub_tok.i)
                    
                    phrase += adj + ' ' + sub_tok.text

                    # save the root word of the word
                    phrase += ' '+token.lemma_ 

                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            
                            # look for object modifier
                            adj = rule2_mod(text,sub_tok.i)
                            
                            phrase += adj+' '+sub_tok.text
                            sent.append(phrase)
            
    return sent

rule1_mod(a) # I/we phrases

[' I get treatment',
 ' surgery reverse colostomy bag',
 ' 5th grader miss graduation',
 ' we have backyard',
 ' We have private office',
 ' me take time',
 ' We have big family',
 ' We do meetings',
 ' We do classes',
 ' We spend lot',
 ' I loose weight',
 ' We do nights',
 ' who take best care',
 ' teachers have meetings',
 ' daughter do classes']

###  Phrases with noun followed by prepositions

In [11]:
# rule 3 function
def rule3(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:

        # look for prepositions
        if token.pos_=='ADP':

            phrase = ''
            
            # if its head word is a noun
            if token.head.pos_=='NOUN':
                
                # append noun and preposition to phrase
                phrase += token.head.text
                phrase += ' '+token.text

                # check the nodes to the right of the preposition
                for right_tok in token.rights:
                    # append if it is a noun or proper noun
                    if (right_tok.pos_ in ['NOUN','PROPN']):
                        phrase += ' '+right_tok.text
                
                if len(phrase)>2:
                    sent.append(phrase)
                
    return sent
rule3(a) # prepositions

['challenges for',
 'beginning of March',
 'diagnosis along',
 'top of',
 'meetings with cousins',
 'lot of time',
 'care of game',
 'meetings with kids']